Get data from github

In [7]:
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_dataset.py

--2023-06-26 08:27:25--  https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 891 [text/plain]
Saving to: ‘bert_dataset.py.1’

bert_dataset.py.1   100%[===================>]     891  --.-KB/s    in 0s      

2023-06-26 08:27:25 (55.4 MB/s) - ‘bert_dataset.py.1’ saved [891/891]



Installing Packages

In [8]:
!pip install sentencepiece==0.1.95

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import pandas as pd

In [11]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
RANDOM_STATE = 12

In [16]:
BACKTEST_CCY = 'USDRUB'
BACKTEST_DAYS = 5
YEARS = [2021, 2022]

PATH_TEXTS = 'drive/MyDrive/data/telegram'
PATH_OPT_PNL = 'drive/MyDrive/data/pnl/option'

N_EPOCHS = 10

In [17]:
import os

# Get all available data_sources
sources = os.listdir(PATH_TEXTS)
sources

['bitkogan.csv',
 'War_Wealth_Wisdom.csv',
 'themovchans.csv',
 'vts.csv',
 'sky_bond.csv',
 'Alfa_Wealth.csv',
 'rshb_invest.csv',
 'headlines_QUANTS.csv',
 'signal.csv',
 'mmi.csv',
 'cbonds.csv']

In [19]:
# Create target variable dataframe
pnl = pd.DataFrame()
for year in YEARS:
    pnl_year = pd.read_csv(f'{PATH_OPT_PNL}/Backtest_{BACKTEST_CCY}_{BACKTEST_DAYS}_days_{year}.txt')
    print(f'{PATH_OPT_PNL}/Backtest_{BACKTEST_CCY}_{BACKTEST_DAYS}_days_{year}.txt')
    pnl = pnl.append(pnl_year)
pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
pnl.dropna(subset=['pnl'], inplace=True)

drive/MyDrive/data/pnl/option/Backtest_USDRUB_5_days_2021.txt


<ipython-input-19-fae68a3a6ca2>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pnl = pnl.append(pnl_year)


drive/MyDrive/data/pnl/option/Backtest_USDRUB_5_days_2022.txt


<ipython-input-19-fae68a3a6ca2>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pnl = pnl.append(pnl_year)


In [20]:
import datetime as dt
from typing import List, Tuple, Union


def _binary_search_by_date(array: List[Tuple[dt.datetime, float]], date_x: dt.datetime) -> Union[int, None]:
    """
    Searches for the index of date_x in the array via binary search.

        Parameters:
            array (list) : A sorted array of (date, float_value) tuples
            date_x (datetime.datetime) : Date to search for

        Returns:
            index_x (int): Index of the searched date in the array.
    """
    left = 0
    right = len(array) - 1

    while left <= right:
        mid = left + (right - left) // 2

        if array[mid - 1][0] <= date_x <= array[mid][0]:
            return mid
        elif date_x > array[mid - 1][0] and date_x > array[mid][0]:
            left = mid + 1
        else:
            right = mid - 1

    return None


def binary_search_time_series(time_series: List[Tuple[dt.datetime, float]], date_start: dt.datetime,
                              date_end: dt.datetime) -> Union[List[Tuple[dt.datetime, float]], None]:
    """
    Searches for the part of the time series that is contained inside [date_start; date_end] period via binary search.

        Parameters:
            time_series (list) : A sorted array of (date, float_value) tuples
            date_start (datetime.datetime) : Starting date of the searched period
            date_end (datetime.datetime) : Ending date of the searched period

        Returns:
            time_series_data (list): Part of the time series that is contained inside [date_start; date_end] period.
    """

    if date_start <= date_end:
        left_index = _binary_search_by_date(time_series, date_start)

        if left_index is None:
            return None

        right_index = _binary_search_by_date(time_series[left_index:], date_end)

        if right_index is None:
            return None

        right_index += left_index
    else:
        left_index = _binary_search_by_date(time_series, date_end)

        if left_index is None:
            return None

        right_index = _binary_search_by_date(time_series[left_index:], date_start)

        if right_index is None:
            return None

        right_index += left_index

    return time_series[left_index:right_index]

In [21]:
pnl_sign_ts = [(pd.to_datetime(row['date_start']).to_pydatetime(), row['pnl']) for  _, row in pnl.iterrows()]
pnl_sign_ts

[(datetime.datetime(2021, 1, 4, 0, 0), 291264.0979135564),
 (datetime.datetime(2021, 1, 5, 0, 0), -411993.8303199067),
 (datetime.datetime(2021, 1, 6, 0, 0), 521491.6867951669),
 (datetime.datetime(2021, 1, 8, 0, 0), 62842.63411628307),
 (datetime.datetime(2021, 1, 11, 0, 0), -537598.7062168794),
 (datetime.datetime(2021, 1, 12, 0, 0), -359059.05784320267),
 (datetime.datetime(2021, 1, 13, 0, 0), 181132.6897838658),
 (datetime.datetime(2021, 1, 14, 0, 0), 47895.30725670268),
 (datetime.datetime(2021, 1, 15, 0, 0), 30290.59373304957),
 (datetime.datetime(2021, 1, 18, 0, 0), 1157975.3869589714),
 (datetime.datetime(2021, 1, 19, 0, 0), 1258409.487166846),
 (datetime.datetime(2021, 1, 20, 0, 0), 1849608.792259631),
 (datetime.datetime(2021, 1, 21, 0, 0), 1996906.8565119945),
 (datetime.datetime(2021, 1, 22, 0, 0), 762996.8728368632),
 (datetime.datetime(2021, 1, 25, 0, 0), 560685.9672308444),
 (datetime.datetime(2021, 1, 26, 0, 0), -95365.18586283142),
 (datetime.datetime(2021, 1, 27, 0, 0

In [22]:
initial_date = pnl_sign_ts[0][0]

def get_pnl_sign(row):
    date_x = pd.to_datetime(row['date']).to_pydatetime()
    pnl_sign_key = _binary_search_by_date(pnl_sign_ts, date_x)

    if pnl_sign_key is not None and date_x >= initial_date:
        return pnl_sign_ts[_binary_search_by_date(pnl_sign_ts, date_x)][1]

In [23]:
# Generate dataframes
df = pd.DataFrame()

for s in sources:
    if s != '.gitignore':
        source_data = pd.read_csv(f'{PATH_TEXTS}/{s}')
        df = df.append(source_data)

df.head()

<ipython-input-23-fe866cb0e0b2>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(source_data)
<ipython-input-23-fe866cb0e0b2>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(source_data)
<ipython-input-23-fe866cb0e0b2>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(source_data)
<ipython-input-23-fe866cb0e0b2>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(source_data)
<ipython-input-23-fe866cb0e0b2>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(source_data)
<ipython-i

,id,date,text
0,3,2018-01-15T16:12:18,из российских биржевых идеек я бы посмотрел се...
1,5,2018-01-16T00:47:25,рекомендовал пару недель назад xmg. cn\nпервые...
2,7,2018-01-16T14:48:39,несмотря на страшно. ужас ужас. выкупаю паден...
3,8,2018-01-16T15:04:23,"январское ралли....в принципе, это не нечто не..."
4,10,2018-01-16T21:36:35,все идет по плану


In [24]:
df['pnl_sign'] = df.apply(lambda row: get_pnl_sign(row), axis=1)
df.dropna(subset=['pnl_sign'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,id,date,text,pnl_sign
0,10186,2021-01-05T09:37:44,Почему просели котировки нефти. Все очень прос...,521491.686795
1,10187,2021-01-05T09:41:54,Brent,521491.686795
2,10188,2021-01-05T09:42:23,Light,521491.686795
3,10189,2021-01-05T10:01:04,По итогам первого торгового дня года получил з...,521491.686795
4,10190,2021-01-05T10:09:56,Что-то непонятное происходит с рублем. USD/RUB...,521491.686795


In [25]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

from functools import lru_cache
from tqdm.notebook import tqdm
from tqdm.gui import tqdm as tqdm_gui

m = MorphAnalyzer()
regex = re.compile("[а-яa-zёЁ]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

mystopwords = stopwords.words('russian')
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)

    return ' '.join(remove_stopwords(lemmas))

In [27]:
lemmas = list(tqdm(map(clean_text, df['text']), total=len(df['text'].to_list())))

  0%|          | 0/45412 [00:00<?, ?it/s]

In [28]:
df['lemmas'] = lemmas
df = df[df['lemmas'] != '']
df.head()

,id,date,text,pnl_sign,lemmas
0,10186,2021-01-05T09:37:44,Почему просели котировки нефти. Все очень прос...,521491.686795,почему просесть котировка нефть очень просто п...
1,10187,2021-01-05T09:41:54,Brent,521491.686795,brent
2,10188,2021-01-05T09:42:23,Light,521491.686795,light
3,10189,2021-01-05T10:01:04,По итогам первого торгового дня года получил з...,521491.686795,итог первый торговый день получить звонок встр...
4,10190,2021-01-05T10:09:56,Что-то непонятное происходит с рублем. USD/RUB...,521491.686795,непонятный происходить рубль взлететь минута п...


In [29]:
df.shape

(45362, 5)

In [30]:
!pip install fast_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
from fast_ml.model_development import train_valid_test_split

X_train, X_train['pnl_sign'], X_valid, X_valid['pnl_sign'], X_test, X_test['pnl_sign'] = train_valid_test_split(df[['lemmas', 'pnl_sign']], target = 'pnl_sign',
                                                                            train_size=0.8, valid_size=0.1, test_size=0.1)

In [54]:
import numpy as np

pnl_mean = X_train['pnl_sign'].mean()
pnl_std = np.std(X_train['pnl_sign'])

X_train['pnl_sign'] = (X_train['pnl_sign'] - pnl_mean) / pnl_std
X_valid['pnl_sign'] = (X_valid['pnl_sign'] - pnl_mean) / pnl_std
X_test['pnl_sign'] = (X_test['pnl_sign'] - pnl_mean) / pnl_std

In [57]:
def classify(data: pd.DataFrame, column_name: str = 'pnl', thr: float = 0.12, include_small: bool = False) -> pd.DataFrame:
    data[column_name] = data[column_name].apply(lambda x: 1 if x >= thr else (0 if x <= -thr else 2))
    if not include_small:
        data = data[data[column_name] != 2]
    return data

In [58]:
X_train = classify(X_train, 'pnl_sign')
X_valid = classify(X_valid, 'pnl_sign')
X_test = classify(X_test, 'pnl_sign')

In [59]:
X_train.shape[0], X_valid.shape[0], X_valid.shape[0]

(30454, 3853, 3853)

In [60]:
X_train['pnl_sign'].sum() / X_train['pnl_sign'].shape[0], X_valid['pnl_sign'].sum() / X_valid.shape[0], X_test['pnl_sign'].sum() / X_test.shape[0]

(0.32071320680370397, 0.3296132883467428, 0.32320586694604503)

Initialize BERT classifier

In [62]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=len(np.unique(X_train['pnl_sign'])),
        epochs=N_EPOCHS,
        model_save_path='/content/bert.pt'
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny a

Prepare data and helpers for train and evlauation

In [63]:
classifier.preparation(
        X_train=X_train['lemmas'].tolist(),
        y_train=X_train['pnl_sign'].tolist(),
        X_valid=X_valid['lemmas'].tolist(),
        y_valid=X_valid['pnl_sign'].tolist()
    )

Train loop

In [64]:
classifier.train()

Epoch 1/10
Train loss 0.7741302063682153 accuracy 0.6718000919419452
Val loss 0.8241888321008191 accuracy 0.6794705424344666
----------
Epoch 2/10
Train loss 0.9145487995735443 accuracy 0.7003020949628949
Val loss 1.0494928335897589 accuracy 0.687516221126395
----------
Epoch 3/10
Train loss 0.9556080950799614 accuracy 0.7184277927365864
Val loss 1.2484328414371009 accuracy 0.6901116013495977
----------
Epoch 4/10
Train loss 0.9465028666651706 accuracy 0.740690878045577
Val loss 1.2164238172524802 accuracy 0.6779133143005449
----------
Epoch 5/10
Train loss 0.92756242060335 accuracy 0.7621330531293098
Val loss 1.3863394824540525 accuracy 0.6823254606799896
----------
Epoch 6/10
Train loss 0.9065254834342845 accuracy 0.779700531949826
Val loss 1.5087198440805194 accuracy 0.6696080975862964
----------
Epoch 7/10
Train loss 0.884316896141067 accuracy 0.7965784461811257
Val loss 1.5239228776754685 accuracy 0.6716844017648584
----------
Epoch 8/10
Train loss 0.8569364706784458 accuracy 0.80

Check test data

In [ ]:
train_accuracies = [0.6718000919419452, 0.7003020949628949, 0.7184277927365864,
                    0.740690878045577, 0.7621330531293098, 0.779700531949826,
                    0.7965784461811257, 0.8083667170158272, 0.8185131674000132,
                    0.8274118342418074]

test_accuracies = [0.6794705424344666, 0.687516221126395, 0.6901116013495977,
                   0.6779133143005449, 0.6823254606799896, 0.6696080975862964,
                   0.6716844017648584, 0.6618219569166882, 0.6610433428497274,
                   0.6641577991175707]

In [65]:
predictions = [classifier.predict(t) for t in X_test['lemmas'].tolist()]

In [66]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(X_test['pnl_sign'].tolist(), predictions, average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.6458699313994729, recall: 0.5658318112709555, f1score: 0.5516176926979139


In [67]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(X_test['pnl_sign'].tolist(), predictions)
cm

array([[2407,  177],
       [ 987,  247]])

In [68]:
accuracy_score(X_test['pnl_sign'].tolist(), predictions)

0.6951283394447355

In [70]:
max(X_test['pnl_sign'].sum() / X_test.shape[0], 1 - X_test['pnl_sign'].sum() / X_test.shape[0])

0.676794133053955

In [71]:
df['lemmas'].apply(lambda x: len(x.split(' '))).mean()

52.61057713504695

In [72]:
12000 / df['lemmas'].apply(lambda x: len(x.split(' '))).mean()

228.09101616956232